This notebook prepares the original dataset for further processing. It uses pyimagej to process the images in a standardized way, saving time and avoiding mistakes.



In [1]:
import os
import shutil
import imagej
import scyjava
from scyjava import jimport
import bioformats
import pandas as pd
import numpy as np

## A. Initialize ImageJ

In [2]:
try: ij
except NameError: ij = imagej.init('sc.fiji:fiji:2.14.0')
# except NameError: ij = imagej.init('2.14.0')
# except NameError: ij = imagej.init('/Applications/Fiji.app/')

11:03:50.943 [SciJava-7c1a88e3-Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
	at loci.formats.ClassList.<init>(ClassList.java:122)
	at loci.formats.ImageReader.getDefaultReaderClasses(ImageReader.java:80)
	at io.scif.bf.BioFormatsFormat.cacheReaderClasses(BioFormatsFormat.java:538)
	at io.scif.bf.BioFormatsFormat.<init>(BioFormatsFormat.java:139)
	at sun.reflect.NativeConstructorAccessorImpl.newInsta

## B. Load data to ImageJ
Load image: ``ij.io().open(path)``

Save image: ``ij.io().save(image, path)``

Get image info:  ``dump_info(image)``

In [3]:
def dump_info(image):
    """A handy function to print details of an image object."""
    name = image.name if hasattr(image, 'name') else None # xarray
    if name is None and hasattr(image, 'getName'): name = image.getName() # Dataset
    if name is None and hasattr(image, 'getTitle'): name = image.getTitle() # ImagePlus
    print(f" name: {name or 'N/A'}")
    print(f" type: {type(image)}")
    print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"shape: {image.shape}")
    print(f" dims: {image.dims if hasattr(image, 'dims') else 'N/A'}")

In [ ]:
## Load java image
# path = '/Users/beatrizfernandes/Documents/Biomédica@IST/Mestrado/PIC2/data/'
# image = ij.io().open(path + 'LMU_29_11_23/duplicated/image_001.tif')
# dump_info(image)

Image from Java to Python: ``ij.py.from_java(image)``

Image from Python to Java: ``ij.py.to_java(image)``

In [ ]:
## Pass image data from Java to Python
# pyimage = ij.py.from_java(image)
# dump_info(pyimage)

## C. Preprocessing

In [4]:
## Define paths
cwd = os.getcwd()
pardir = os.path.dirname(os.path.dirname(cwd))
path_data = pardir + "/data/preprocessing_auto"
path_macros = pardir + "/macros"
path_originals = path_data + "/originals/"
path_auxiliary = '/Users/beatrizfernandes/Documents/Biomédica@IST/Mestrado/PIC2/auxiliary/'

path_split = path_data + "/layer 0/split channels dapi-gfap/"
if not os.path.exists(path_split):
    os.makedirs(path_split)
path_normalized = path_data + "/layer 1/normalized/"
# path_contrast = path_data + "/layer 1/contrast/"
path_mean = path_data + "/layer 1/filter mean/"
path_gaussian = path_data + "/layer 1/filter gaussian/"
# path_gaussian3d = path_data + "/layer 1/filter gaussian 3D/"
if not os.path.exists(path_normalized):
    os.makedirs(path_normalized)
# if not os.path.exists(path_contrast):
#     os.makedirs(path_contrast)
if not os.path.exists(path_mean):
    os.makedirs(path_mean)
if not os.path.exists(path_gaussian):
    os.makedirs(path_gaussian)
# if not os.path.exists(path_gaussian3d):
#     os.makedirs(path_gaussian3d)

path_composite = path_data + "/layer 1/composite/"
path_rgb = path_data + "/layer 2/images rgb/"
path_maximum = path_data + "/layer 2/maximum projections/"
path_nuclei2d = path_data + "/layer 2/nuclei 2d/"
path_nuclei3d = path_data + "/layer 2/nuclei 3d/"
if not os.path.exists(path_composite):
    os.makedirs(path_composite)
if not os.path.exists(path_rgb):
    os.makedirs(path_rgb)
if not os.path.exists(path_maximum):
    os.makedirs(path_maximum)
if not os.path.exists(path_nuclei2d):
    os.makedirs(path_nuclei2d)
if not os.path.exists(path_nuclei3d):
    os.makedirs(path_nuclei3d)

In [1]:
def prep_macros():
    """close all windows in ImageJ background and created a dummy image necessary for the package"""
    ij.py.run_macro('run("Close All")')
    if ij.WindowManager.getIDList() is None:
        ij.py.run_macro('newImage("dummy", "8-bit", 1, 1, 1);')

### 1. Duplicate all dataset

In [ ]:
prep_macros()

dataset_list = os.listdir(path_originals)
image_ids = ["%03d" % i for i in range(1, len(dataset_list)+1)]

macro_duplicate_split = open(file = path_macros + "/duplicate_split.ijm", mode='r').read()

for im, image_name in enumerate(list(filter(lambda element: '.tif' in element, os.listdir(path_originals)))):
    if image_name.split('.')[-1]=="tif":
        image_path = path_originals + image_name
        image_id = image_ids[im]

        print(image_path)
        
        # image = ij.io().open(image_path)
        # image_plus = ij.py.to_imageplus(image)
        # image_plus.setTitle("image_" + image_id)
        # image_plus.show()
        args = {
            'image_path': image_path,
            'image_id': image_id,
            'path_split': path_split
        }
        ij.py.run_macro(macro_duplicate_split, args)
        print("done " + image_id)
        ij.window().clear()

### (Opt.). Fluorescence correction

In [ ]:
# prep_macros()

# macro_correction = open(file = path_macros + "/correction.ijm", mode='r').read()

# for image_name in list(filter(lambda element: '.tif' in element, os.listdir(path_split))):
#     image_name = image_name.split(".")[0]

#     f_mean_slices = "mean slices " + image_name.split('_')[2] + '.csv'
#     print(image_name.split('_')[2])
#     if image_name.split('_')[2] == 'dapi':
#         m, b = (-0.1339, 1.6732)
#     else:
#         m, b = (-0.3214, 2.4512)
    
#     mean_slices = pd.read_csv(path_auxiliary + f_mean_slices, delimiter=';', index_col='slice')
#     mean_min = mean_slices[image_name].min()
#     print(mean_min)

#     args ={
#         'image_name': image_name,
#         'mean_min': str(np.round(mean_min,3)),
#         'path_corrected': path_corrected,
#         'm': str(m),
#         'b': str(b),
#     }
#     ij.py.run_macro(macro_correction, args)
#     print(path_split + image_name)
#     ij.window().clear()

### 2. Normalization

In [ ]:
for image_name in np.sort(list(filter(lambda element: '.tif' in element, os.listdir(path_split)))):
    java_image = ij.io().open(path_split + image_name)
    py_image = ij.py.from_java(java_image)
    if py_image.min() != 0 or py_image.max() != 255:
        py_image = (py_image-py_image.min())/(py_image.max()-py_image.min())*255
    java_image = ij.py.to_java(py_image.astype(np.uint8))
    ij.io().save(java_image, path_normalized + image_name.split('.')[0] + '_normalized.tif')

### 3. Contrast & Filtering

In [ ]:
prep_macros()

macro_filtering = open(file = path_macros + "/filtering.ijm", mode='r').read()

for image_name in list(filter(lambda element: '.tif' in element, os.listdir(path_normalized))):
    image_name = '_'.join(image_name.split("_")[:3])
    print(image_name)

    if image_name.split('_')[2] == 'dapi':
        color = 'Magenta'
    else: 
        color = 'Green'

    args ={
        'image_name': image_name,
        'image_path': path_normalized,
        #'path_contrast': path_contrast,
        'path_mean': path_mean,
        'path_gaussian': path_gaussian,
        #'path_gaussian3d': path_gaussian3d,
        'color': color
    }
    ij.py.run_macro(macro_filtering, args)
    print(path_split + image_name)
    ij.window().clear()

### 4. Datasets for Image Analysis

In [8]:
prep_macros()

macro_datasets = open(file = path_macros + "/datasets.ijm", mode='r').read()
layer2_source = path_mean

image_ids = []
for i in os.listdir(layer2_source): image_ids.append(i.split("_")[1])
image_ids = list(set(image_ids))

image_names = os.listdir(layer2_source)

for image_id in image_ids:
    files = list(filter(lambda element: image_id in element, os.listdir(layer2_source)))
    print(files)
    name_dapi = list(filter(lambda element: 'dapi' in element, files))[0].split('.')[0]
    name_gfap = list(filter(lambda element: 'gfap' in element, files))[0].split('.')[0]

    args ={
        'images_path': layer2_source,
        'image_id': 'image_' + image_id,
        'name_dapi': name_dapi,
        'name_gfap': name_gfap,
        'path_composite': path_composite,
        'path_rgb': path_rgb,
        'path_maximum': path_maximum,
        'path_nuclei2d': path_nuclei2d,
        'path_nuclei3d': path_nuclei3d
    }
    ij.py.run_macro(macro_datasets, args)
    print(image_id + ' done')
    ij.window().clear()

Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the WindowManager class will not be fully functional.


['image_004_gfap_mean.tif', 'image_004_dapi_mean.tif']
java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpreter.java:1

[java.lang.Enum.toString] java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpreter.java:107)
	at ij.plugin.Macro_Runne

004 done
['image_007_gfap_mean.tif', 'image_007_dapi_mean.tif']
java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpret

[java.lang.Enum.toString] java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpreter.java:107)
	at ij.plugin.Macro_Runne

...

[java.lang.Enum.toString] java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpreter.java:107)
	at ij.plugin.Macro_Runne

014 done
['image_013_gfap_mean.tif', 'image_013_dapi_mean.tif']
java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpret

[java.lang.Enum.toString] java.lang.NullPointerException
	at ij.plugin.RGBStackConverter.dialogItemChanged(RGBStackConverter.java:275)
	at net.imagej.patcher.HeadlessGenericDialog.showDialog(HeadlessGenericDialog.java:237)
	at ij.plugin.RGBStackConverter.showDialog(RGBStackConverter.java:262)
	at ij.plugin.RGBStackConverter.compositeToRGB(RGBStackConverter.java:93)
	at ij.plugin.RGBStackConverter.run(RGBStackConverter.java:38)
	at ij.IJ.runPlugIn(IJ.java:210)
	at ij.Executer.runCommand(Executer.java:152)
	at ij.Executer.run(Executer.java:70)
	at ij.IJ.run(IJ.java:325)
	at ij.IJ.run(IJ.java:336)
	at ij.macro.Functions.doRun(Functions.java:703)
	at ij.macro.Functions.doFunction(Functions.java:99)
	at ij.macro.Interpreter.doStatement(Interpreter.java:281)
	at ij.macro.Interpreter.doStatements(Interpreter.java:267)
	at ij.macro.Interpreter.run(Interpreter.java:163)
	at ij.macro.Interpreter.run(Interpreter.java:93)
	at ij.macro.Interpreter.run(Interpreter.java:107)
	at ij.plugin.Macro_Runne

013 done


Macro Error: ')' expected in line 42
 
run ( "Bio-Formats Exporter" , "save=[path_nuclei2d + " <bio> " + image_id + " _nuclei2d_bio . png ] ");" [java.lang.Enum.toString] Macro Error: ')' expected in line 42
 
run ( "Bio-Formats Exporter" , "save=[path_nuclei2d + " <bio> " + image_id + " _nuclei2d_bio . png ] ");" 
[java.lang.Enum.toString] 


### 5. Analysis

In [ ]:
path_output = path_data + "/results/"

In [ ]:
prep_macros()

macro_results = open(file = path_macros + "/results.ijm", mode='r').read()
path_input = path_normalized
path_results = path_data + "/results/"
if not os.path.exists(path_results):
    os.makedirs(path_results)

for image_name in list(filter(lambda element: '.tif' in element, os.listdir(path_input))):
    image_name = image_name.split(".")[0]
    print(image_name)

    args ={
        'image_name': image_name,
        'path_input': path_input,
        'path_results': path_results
    }
    ij.py.run_macro(macro_results, args)
    print(path_split + image_name)
    ij.window().clear()

## D. Datasets for Segmentation

### 1. Dataset for Instance Segmentation with InstantDL

#### 2D

In [9]:
path_input = path_data + "/layer 2/nuclei 2d bio/"
path_roi2d = path_data + "/rois_nuclei2d/"
path_output_instance = path_data + "/layer 3/2d instance bio masks/"
if not os.path.exists(path_output_instance):
    os.makedirs(path_output_instance)

In [10]:
macro_instance_segmentation = open(file = path_macros + "/instance_segmentation.ijm", mode='r').read()

for image_id in list(filter(lambda element: 'image' in element, os.listdir(path_input))):
    # image folder
    f_image_path = path_output_instance + image_id.split('.')[0] + "/image/"
    if not os.path.exists(f_image_path):
        os.makedirs(f_image_path)
    image_path = f_image_path + image_id
    shutil.copyfile(path_input + image_id, image_path)
    # mask folder
    mask_path = path_output_instance + image_id.split('.')[0] + "/mask/"
    if not os.path.exists(mask_path):
        os.makedirs(mask_path)
    for roi_id, roi_name in enumerate(os.listdir(path_roi2d + image_id.split('.')[0] + "_RoiSet/")):  #.split('-')[0]
        args = {
            'image_path': image_path,
            'image_id': image_id.split('.')[0],
            'mask_path': mask_path,
            'roi_path': path_roi2d + image_id.split('.')[0]+ "_RoiSet/" + roi_name,  #.split('-')[0]
            'roi_id': roi_id
        }
        print(path_roi2d + image_id.split('-')[0]+ "_RoiSet/" + roi_name)
        ij.py.run_macro(macro_instance_segmentation, args)
        ij.window().clear()
    print(image_id + ' done')

/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0814-0057.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0188-0632.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0352-0644.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0598-0567.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0363-0665.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0675-0464.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0783-0388.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0089-0502.roi
/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei2d/image_019_nuclei2d.png_RoiSet/0633-0278.roi
/

#### 3D

In [ ]:
path_input = path_data + "/layer 2/nuclei 3d/"
path_roi3d = path_data + "/rois_nuclei3d/"
path_output_instance_3d = path_data + "/layer 3/3d instance masks/"
if not os.path.exists(path_output_instance_3d):
    os.makedirs(path_output_instance_3d)

In [ ]:
macro_make_stack = """ 
#@ String path
#@ String mask_id

newImage("HyperStack", "8-bit color-mode", 1024, 1024, 1, 64, 1);
run("Hyperstack to Stack");
saveAs("Tiff", path + mask_id + ".tif");
run("Close All");
"""

macro_make_mask = """
#@ String rois_path
#@ String roi_id
#@ String path_mask
#@ String mask_id
#@ Integer slice

print(rois_path + roi_id + ".roi")
open(path_mask + mask_id + ".tif");
open(rois_path + roi_id + ".roi");
selectImage(mask_id + ".tif");
run("Create Mask");
rename("Mask");
selectImage(mask_id + ".tif");
setSlice(slice);
run("Add Image...", "image=Mask x=0 y=0 opacity=100");
selectImage(mask_id + ".tif");
saveAs("Tiff", path_mask + mask_id + ".tif");
run("Close All");
"""

In [ ]:
## set input folder 
for rois3d in list(filter(lambda element: '.zip' not in element and 'image' in element, os.listdir(path_roi3d))):
    image_id = rois3d[:-7]
    rois = list(filter(lambda element: 'cell' in element, os.listdir(path_roi3d + rois3d)))
    print(path_roi3d + rois3d)
    print(image_id)
    cell_ids = []
    args = {'file': path_nuclei3d + image_id,
            'image_id': image_id}
    for i in rois: 
        cell_ids.append(i.split('_')[1])
    cell_ids = list(set(cell_ids))
    print(cell_ids)
    for cell_id in cell_ids:
        if not os.path.exists(path_roi3d + rois3d + '/' + image_id + '_ind' + str(int(cell_id)-1)):
            os.mkdir(path_roi3d + rois3d + '/' + image_id + '_ind' + str(int(cell_id)-1))
        for roi in list(filter(lambda element: cell_id in element, os.listdir(path_roi3d + rois3d))):
            shutil.move(path_roi3d + rois3d + '/' + roi, path_roi3d + rois3d + '/' + image_id + '_ind' + str(int(cell_id)-1))
            # shutil.make_archive(path_roi3d + rois3d + '/' + cell_id, 'zip')

In [ ]:
for rois3d in list(filter(lambda element: '.zip' not in element and 'image' in element, os.listdir(path_roi3d))):
    image_id = rois3d[:-7]
    f_image_path = path_output_instance_3d + image_id + "/image/"
    if not os.path.exists(f_image_path):
        os.makedirs(f_image_path)
    image_path = f_image_path + image_id + '.tif'
    shutil.copyfile(path_input + image_id + '.tif', image_path)
    # mask folder
    mask_path = path_output_instance_3d + image_id + "/mask/"
    if not os.path.exists(mask_path):
        os.makedirs(mask_path)

    for mask_id in list(filter(lambda element: 'image' in element, os.listdir(path_roi3d + rois3d))):
        args = {
                'path': mask_path,
                'mask_id': mask_id
                }
        ij.py.run_macro(macro_make_stack, args)
        for roi in list(filter(lambda element: '.roi' in element, os.listdir(path_roi3d + rois3d + '/' + mask_id))):
            print(roi)
            slice = int(roi.split('_')[2].split('.')[0])
            # print(path_roi3d + rois3d + '/' + mask_id + '/' + roi)
            args = {
                'rois_path': path_roi3d + rois3d + '/' + mask_id + '/',
                'roi_id': roi.split('.')[0],
                'path_mask': mask_path,
                'mask_id': mask_id,
                'slice': slice,
                }
            ij.py.run_macro(macro_make_mask, args)

        ij.py.run_macro("""run("Close All")""")

In [ ]:
# macro_open_image = """
# #@ String file
# #@ String image_id
# open(file + ".tif");
# selectImage(image_id + ".tif");
# """
# macro_add_rois = """ 
# #@ String file
# roiManager("Open", file);
# """
# macro_save_mask = """
# #@ String directory
# #@ String cell_id
# #@ String size
# run("Binary (0-255) mask(s) from Roi(s)", "show_mask(s) save_mask(s) save_in=directory suffix=cell_id save_mask_as=tif rm=[RoiManager[size=size, visible=true]]");
# roiManager("Delete");
# run("Close All)
# """
# for rois3d in list(filter(lambda element: '.zip' not in element and 'image' in element, os.listdir(path_roi3d))):
#     image_id = rois3d[:-7]
#     rois = list(filter(lambda element: 'cell' in element, os.listdir(path_roi3d + rois3d)))
#     print(path_roi3d + rois3d)
#     print(rois)
#     cell_ids = []
#     args = {'file': path_nuclei3d + image_id,
#             'image_id': image_id}
#     ij.py.run_macro(macro_open_image, args)
#     for i in rois: 
#         cell_ids.append(i.split('_')[1])
#     cell_ids = list(set(cell_ids))
#     print(rois)
#     for cell_id in cell_ids:
#         if not os.path.exists(path_roi3d + rois3d + '/' + cell_id):
#             os.mkdir(path_roi3d + rois3d + '/' + cell_id)
#         for roi in list(filter(lambda element: cell_id in element, os.listdir(path_roi3d + rois3d))):
#             shutil.move(path_roi3d + rois3d + '/' + roi, path_roi3d + rois3d + '/' + cell_id)
#             shutil.make_archive(path_roi3d + rois3d + '/' + cell_id, 'zip')
#             print(cell_id)
#         for roi_id in os.listdir(path_roi3d + rois3d + '/' + cell_id):
#             args = {'file': path_roi3d + rois3d + '/' + cell_id + '/' + roi_id}
#             ij.py.run_macro(macro_add_rois, args)
#         size = len(os.listdir(path_roi3d + rois3d + '/' + cell_id))
#         args = {'directory': path_output_instance,
#                 'cell_id': cell_id,
#                 'size': size}
#         ij.py.run_macro(macro_save_mask, args)

### 2. Dataset for Semantic Segmentation with InstantDL

#### 2D

In [ ]:
path_input = path_data + "/layer 2/nuclei 2d/"
path_roi2d = path_data + "/rois_nuclei2d/"
path_output_semantic = path_data + "/layer 3/semantic masks/"
if not os.path.exists(path_output_semantic):
    os.makedirs(path_output_semantic)

In [ ]:
macro_semantic_segmentation = open(file = path_macros + "/semantic_segmentation.ijm", mode='r').read()
image_path = path_output_semantic + "image/"
groundtruth_path = path_output_semantic + "groundtruth/"
if not os.path.exists(image_path):
    os.makedirs(image_path)
if not os.path.exists(groundtruth_path):
    os.makedirs(groundtruth_path)

for image_id in list(filter(lambda element: 'image' in element, os.listdir(path_input))):
    # copy image
    shutil.copyfile(path_input + image_id, image_path + image_id)
    args = {
        'image_path': image_path,
        'image_id': image_id,
        'groundtruth_path': groundtruth_path,
        'roi_path': path_roi2d + image_id.split('.')[0]+ "_RoiSet.zip",
    }
    ij.py.run_macro(macro_semantic_segmentation, args)
    ij.window().clear()  
    print(image_id + ' done')

## E. 3D Intance Segmentation Folders

In [ ]:
import shutil

In [ ]:
rois = list(filter(lambda element: 'cell' in element, os.listdir(path_roi3d + rois3d)))
rois

In [ ]:
list(filter(lambda element: '.zip' not in element and 'image' in element, os.listdir(path_roi3d)))[0][:-7]

In [ ]:
os.listdir('/Users/beatrizfernandes/PIC2/data/preprocessing_auto/rois_nuclei3d/copy')

for cell_id in list(filter(lambda element: 'cell' in element, os.listdir(path_roi3d)))